<a href="https://colab.research.google.com/github/faranak5171/Exploratory-Data-Analysis-sql/blob/main/International%20debt/international_debt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install db-sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 171 kB 31.0 MB/s 
  Created wheel for db-sqlite3: filename=db_sqlite3-0.0.1-py3-none-any.whl size=1794 sha256=10fb11dfc7af7c2b51cc9001f2c96235c471ad0f7a307316d43e0fc24816758c
  Stored in directory: /root/.cache/pip/wheels/02/38/d5/2f54461050571bf5330fee2a37ab1c9b5e7540b0572f1acdab
  Created wheel for db: filename=db-0.1.1-py3-none-any.whl size=3895 sha256=38f9f592047bae5e43279796c2017d53150d47d41a2262cd1e0ace8d8585cc95
  Stored in directory: /root/.cache/pip/wheels/8e/97/82/741d2b360507411ec233d0280d7371faa94b03bde834e4a9be
  Created wheel for antiorm: filename=antiorm-1.2.1-py3-none-any.whl size=31679 sha256=7222c843143ba0a46adee460529d26ddd6e1199baee0c39086eef3861cf7933d
  Stored in directory: /root/.cache/pip/wheels/c5/43/70/e9729370cfff40c49d3e3d05377d54b3ecd71f64e62341ea80
Successfully built db-sqlite3 db antiorm


In [2]:
import sqlite3
import pandas as pd

In [3]:
db = sqlite3.connect("InterDebtDB.db")
db.execute("drop table if exists international_debt")
try:
  db.execute("CREATE TABLE international_debt (country_name character varying(50), country_code character varying(50), indicator_name text, indicator_code text, debt numeric)")
except:
  print("Already table existed !")

In [5]:
df = pd.read_csv("international_debt.csv")
df.head()

,country_name,country_code,indicator_name,indicator_code,debt
0,Afghanistan,AFG,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,72894453.7
1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,53239440.1
2,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.9
3,Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.4
4,Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.1


In [6]:
for i in range(df.shape[0]):
  if "'" in df.iloc[i]["country_name"]:
    country_name = str(df.iloc[i]["country_name"]).replace("'","''")
  else:
    country_name = str(df.iloc[i]["country_name"])
  cmd = "INSERT INTO international_debt (country_name,country_code,indicator_name, indicator_code, debt) VALUES ('{}','{}','{}','{}','{}')".format(country_name, df.iloc[i]["country_code"], df.iloc[i]["indicator_name"], df.iloc[i]["indicator_code"],df.iloc[i]["debt"])
  db.execute(cmd)
  db.commit()

In [7]:
#SELECT all of the columns from the international_debt table . limit the output to the first ten rows to keep the output clean.
cmd = "SELECT * FROM international_debt limit 10;"
df_db = pd.read_sql_query(cmd,db)
df_db.head(10)

,country_name,country_code,indicator_name,indicator_code,debt
0,Afghanistan,AFG,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,72894453.7
1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,53239440.1
2,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.9
3,Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.4
4,Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.1
5,Afghanistan,AFG,"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD,39107845.0
6,Afghanistan,AFG,"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD,23779724.3
7,Afghanistan,AFG,"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD,13335820.0
8,Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,100847181.9
9,Afghanistan,AFG,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,72894453.7


In [8]:
#Finding the number of distinct countries
cmd = "SELECT count(distinct country_name) as total_countries FROM international_debt;"
df_db = pd.read_sql_query(cmd,db)
df_db.head()

,total_countries
0,124


In [9]:
#Finding out the distinct debt indicators
cmd = "SELECT count(distinct indicator_code) as total_indicators FROM international_debt;"
df_db = pd.read_sql_query(cmd,db)
df_db.head()

,total_indicators
0,25


In [10]:
#Totaling the amount of debt owed by the countries
cmd = "SELECT CAST(sum(debt) as INT) as sum_debt FROM international_debt;"
df_db = pd.read_sql_query(cmd,db)
df_db.head()

,sum_debt
0,3079734487692


In [11]:
# Country with the highest debt
cmd = "SELECT CAST(sum(debt) as INT) as sum_debt , country_name FROM international_debt GROUP BY country_name ORDER BY sum_debt DESC;"
df_db = pd.read_sql_query(cmd,db)
df_db.head(10)

,sum_debt,country_name
0,285793494734,China
1,280623966141,Brazil
2,247608723991,South Asia
3,212880992793,Least developed countries: UN classification
4,191289057259,Russian Federation
5,179048127208,IDA only
6,151125758036,Turkey
7,133627060958,India
8,124596786218,Mexico
9,113435696695,Indonesia


In [13]:
#Average amount of debt across indicators
cmd = "SELECT round(avg(debt),2) AS average_debt, indicator_code FROM international_debt GROUP BY indicator_code ORDER BY average_debt DESC;"
df_db = pd.read_sql_query(cmd,db)
df_db.head()

,average_debt,indicator_code
0,5.904868e+09,DT.AMT.DLXF.CD
1,5.161194e+09,DT.AMT.DPNG.CD
2,2.152041e+09,DT.DIS.DLXF.CD
3,1.958983e+09,DT.DIS.OFFT.CD
4,1.803694e+09,DT.AMT.PRVT.CD


In [14]:
# Which country owes the highest amount of debt in the category of long term debts (DT.AMT.DLXF.CD).
cmd = "SELECT country_name, indicator_name FROM international_debt WHERE debt = (SELECT max(debt) FROM international_debt where indicator_code='DT.AMT.DLXF.CD');"
df_db = pd.read_sql_query(cmd,db)
df_db.head()

,country_name,indicator_name
0,China,"Principal repayments on external debt, long-te..."


In [15]:
# The most common debt indicator
cmd = "SELECT count(indicator_code) as indicator_count,indicator_code FROM international_debt GROUP BY indicator_code ORDER BY indicator_count DESC LIMIT 20;"
df_db = pd.read_sql_query(cmd,db)
df_db.head()

,indicator_count,indicator_code
0,124,DT.AMT.DLXF.CD
1,124,DT.AMT.MLAT.CD
2,124,DT.AMT.OFFT.CD
3,124,DT.INT.DLXF.CD
4,124,DT.INT.MLAT.CD


In [16]:
# Maximum amount of debt that each country owes.
cmd = "SELECT country_name, max(debt) as maximum_debt FROM international_debt GROUP BY country_name ORDER BY maximum_debt DESC LIMIT 10;"
df_db = pd.read_sql_query(cmd,db)
df_db.head()

,country_name,maximum_debt
0,China,96218620836
1,Brazil,90041840304
2,Russian Federation,66589761834
3,Turkey,51555031006
4,South Asia,48756295898
